In [1]:
import cobra
import pandas as pd
import numpy as np
import os
import sys
import pickle
from tools import save_pickle_model

In [2]:
ecpy_path = '../../../ecpy/'
sys.path.append(os.path.abspath(ecpy_path))
import utils
import ecpy

In [3]:
import importlib
importlib.reload(utils)

<module 'utils' from '/Users/gangl/Documents/GitHub/Halo-GEM/ecpy/utils.py'>

#### 1. Add PHA synthesis reaction
(R)-3-Hydroxybutanoyl-CoA  -> CoA + PHA  
 

|Name                     |MetaCyc ID |
|:------------------------|-----------|
|(R)-3-Hydroxybutanoyl-CoA|CPD-650    |
|CoA                      |CO-A       |

The reactions is catalyzed by phaC gene. It has two isoforms: **TD01GL001170**, **TD01GL001573**. The first gene palys the major rule in PHA synthesis

In [4]:
def add_PHA(model_file):
    model = pickle.load(open(model_file,'rb'))
    met_cpd = model.metabolites.get_by_id('CPD-650_c')
    for rxn in met_cpd.reactions: print(rxn.id, rxn.reaction,rxn.gene_reaction_rule)
        
    rxn_pha = cobra.Reaction('PHA_synthetase',name='Poly(3-hydroxyalkanoate) synthetase')
    met_pha = cobra.Metabolite('PHA_c',name='PHA',compartment='c')
    rxn_pha.add_metabolites({
                                met_cpd                              :-1,
                                model.metabolites.get_by_id('CO-A_c'): 1,
                                met_pha                              : 1,
                            })
    rxn_pha.annotation = {'ec-code':'2.3.1-'} # from kegg annotation 
    rxn_pha.gene_reaction_rule = 'TD01GL001170 or TD01GL001573'

    rxn_pha_sec = cobra.Reaction('PHA_secretion',name='PHA secretion')
    rxn_pha_sec.add_metabolites({met_pha:-1})
    
    print(rxn_pha.reaction)
    print(rxn_pha_sec.reaction)
    model.add_reactions([rxn_pha,rxn_pha_sec])
    
    utils.set_yeast_extraction(model,lb=0,ub=0)
    model.objective='PHA_secretion'
    model.objective_direction = 'max'
    s = model.optimize()
    
    print('Maximizing PHA')
    print(s)
    
    
    ## add gene name
    utils.correct_genename(model,'TD01GL001170','phaC1')
    utils.correct_genename(model,'TD01GL001573','phaC2')
    
    # phaB
    utils.print_reactions_of_gene(model,'TD01GL000448')
    utils.correct_genename(model,'TD01GL000448','phaB1')
    
    # phaA3, only correct gene name for this one
    utils.print_reactions_of_gene(model,'TD01GL000367')
    utils.correct_genename(model,'TD01GL000367','phaA3')
    
    return model

#### 2. Add Non-Growth Associated ATP maitanance reaction
1 ATP + 1 H2o --> 1 ADP + 1Pi + 1 H+  
Set the lower bound of the reaction as estimated NGAM, use the value from iML1515 model.

In the model, there will be two reactions for ATP hydrolysis, one with gene reaction rule with a lower bound of 0 and the other without gene reaction rule for NGAM.

In [5]:
def add_NGAM(halo_model):
    utils.test_biomass_production(halo_model)
    ngam_rxn = cobra.Reaction('NGAM')
    ngam_rxn.add_metabolites({
        halo_model.metabolites.ATP_c  :-1,
        halo_model.metabolites.get_by_id('CPD-15815_c'):-1,
        halo_model.metabolites.ADP_c  :1,
        halo_model.metabolites.Pi_c   : 1,
        halo_model.metabolites.PROTON_c:1,
    })

    ngam_rxn.lower_bound = 6.86
    ngam_rxn.upper_bound = 6.86
    ngam_rxn.name = 'Non-Growth Associated ATP maintenance (NGAM)'
    print(ngam_rxn)
    halo_model.add_reactions([ngam_rxn])
    halo_model.repair()
    return halo_model

In [6]:
def run_pipeline(model_file,outfile):
    model = add_PHA(model_file)
    model = add_NGAM(model)
    save_pickle_model(model,outfile)


In [7]:
run_pipeline('../Results/halo_metacycmodel_100_curated_compart_with_eco_medium_biomass_gapfilled.pkl',
             '../Results/halo_metacycmodel_100_curated_compart_with_eco_medium_biomass_gapfilled_PHA_NGAM.pkl')

Using license file /Users/gangl/gurobi.lic
Academic license - for non-commercial use only
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmph8p6bguu.lp
Reading time = 0.03 seconds
: 2400 rows, 4670 columns, 22078 nonzeros
RXN-5901 ACETOACETYL-COA_c + NADPH_c + PROTON_c --> CPD-650_c + NADP_c TD01GL000448 or TD01GL001906
3-HYDROXBUTYRYL-COA-DEHYDRATASE-RXN CPD-650_c <=> CPD-15815_c + CROTONYL-COA_c TD01GL003540
RXN-14255 CPD-15815_c + CPD-650_c --> CO-A_c + CPD-335_c + PROTON_c TD01GL003102
CPD-650_c --> CO-A_c + PHA_c
PHA_c --> 
Maximizing PHA
<Solution 0.832 at 0x131fa3fcc0>
TD01GL000448 or TD01GL001906
Acetoacetyl-CoA + Nicotinamide adenine dinucleotide phosphate - reduced + H+ --> (3R)-3-hydroxybutanoyl-CoA + Nicotinamide adenine dinucleotide phosphate

TD01GL000448
(3R)-3-hydroxypentanoyl-CoA + Nicotinamide adenine dinucleotide phosphate --> 3-oxopentanoyl-CoA + Nicotinamide adenine dinucleotide phosphate - reduced + H+

TD01GL001300 or TD01GL001318

In [8]:
run_pipeline('../Results/halo_metacyc_kegg_curated_compart_with_eco_medium_biomass_gapfilled.pkl',
             '../Results/halo_metacyc_kegg_curated_compart_with_eco_medium_biomass_gapfilled_PHA_NGAM.pkl')

Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpvcwj7mij.lp
Reading time = 0.02 seconds
: 3311 rows, 6348 columns, 29580 nonzeros
5.1.2.3-RXN S-3-HYDROXYBUTANOYL-COA_c <=> CPD-650_c TD01GL001327
3-HYDROXBUTYRYL-COA-DEHYDRATASE-RXN CPD-650_c <=> CPD-15815_c + CROTONYL-COA_c TD01GL003540
RXN-5901 ACETOACETYL-COA_c + NADPH_c + PROTON_c --> CPD-650_c + NADP_c TD01GL000448 or TD01GL001906
RXN-14255 CPD-15815_c + CPD-650_c --> CO-A_c + CPD-335_c + PROTON_c TD01GL003102
CPD-650_c --> CO-A_c + PHA_c
PHA_c --> 
Maximizing PHA
<Solution 1.200 at 0x132428c780>
TD01GL000448
(3R)-3-hydroxypentanoyl-CoA + Nicotinamide adenine dinucleotide phosphate --> 3-oxopentanoyl-CoA + Nicotinamide adenine dinucleotide phosphate - reduced + H+

TD01GL000448 or TD01GL002267 or TD01GL002055
Nicotinamide adenine dinucleotide phosphate - reduced + H+ + Very-Long-Chain-oxoacyl-CoAs --> Nicotinamide adenine dinucleotide phosphate + Very-Long-Chain-3-Hydroxyacyl-CoAs

TD01GL000448 or 

In [9]:
run_pipeline('../Results/halo_metacycmodel_100_curated_compart_with_eco_without_missing_medium_biomass_gapfilled.pkl',
             '../Results/halo_metacycmodel_100_curated_compart_with_eco_without_missing_medium_biomass_gapfilled_PHA_NGAM.pkl')

Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp_hn7_9wh.lp
Reading time = 0.02 seconds
: 2338 rows, 4532 columns, 21360 nonzeros
RXN-5901 ACETOACETYL-COA_c + NADPH_c + PROTON_c --> CPD-650_c + NADP_c TD01GL000448 or TD01GL001906
RXN-14255 CPD-15815_c + CPD-650_c --> CO-A_c + CPD-335_c + PROTON_c TD01GL003102
3-HYDROXBUTYRYL-COA-DEHYDRATASE-RXN CPD-650_c <=> CPD-15815_c + CROTONYL-COA_c TD01GL003540
CPD-650_c --> CO-A_c + PHA_c
PHA_c --> 
Maximizing PHA
<Solution 0.581 at 0x13255db668>
TD01GL000448 or TD01GL001906
Acetoacetyl-CoA + Nicotinamide adenine dinucleotide phosphate - reduced + H+ --> (3R)-3-hydroxybutanoyl-CoA + Nicotinamide adenine dinucleotide phosphate

TD01GL000448
(3R)-3-hydroxypentanoyl-CoA + Nicotinamide adenine dinucleotide phosphate --> 3-oxopentanoyl-CoA + Nicotinamide adenine dinucleotide phosphate - reduced + H+

TD01GL001300 or TD01GL001318 or TD01GL001328 or TD01GL000367
2.0 Acetyl-CoA <=> Acetoacetyl-CoA + Coenzyme A

NGAM: AT

In [10]:
run_pipeline('../Results/halo_metacyc_kegg_curated_compart_with_eco_without_missing_medium_biomass_gapfilled.pkl',
             '../Results/halo_metacyc_kegg_curated_compart_with_eco_without_missing_medium_biomass_gapfilled_PHA_NGAM.pkl')

Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpgc1h2yky.lp
Reading time = 0.02 seconds
: 3253 rows, 6194 columns, 28780 nonzeros
3-HYDROXBUTYRYL-COA-DEHYDRATASE-RXN CPD-650_c <=> CPD-15815_c + CROTONYL-COA_c TD01GL003540
RXN-14255 CPD-15815_c + CPD-650_c --> CO-A_c + CPD-335_c + PROTON_c TD01GL003102
RXN-5901 ACETOACETYL-COA_c + NADPH_c + PROTON_c --> CPD-650_c + NADP_c TD01GL000448 or TD01GL001906
5.1.2.3-RXN S-3-HYDROXYBUTANOYL-COA_c <=> CPD-650_c TD01GL001327
CPD-650_c --> CO-A_c + PHA_c
PHA_c --> 
Maximizing PHA
<Solution 1.200 at 0x13236935f8>
TD01GL000448 or TD01GL002267 or TD01GL002055
Nicotinamide adenine dinucleotide phosphate - reduced + H+ + Very-Long-Chain-oxoacyl-CoAs --> Nicotinamide adenine dinucleotide phosphate + Very-Long-Chain-3-Hydroxyacyl-CoAs

TD01GL000448 or TD01GL001906
Acetoacetyl-CoA + Nicotinamide adenine dinucleotide phosphate - reduced + H+ --> (3R)-3-hydroxybutanoyl-CoA + Nicotinamide adenine dinucleotide phosphate

TD01